# Camera geometry
The purpose of this notebook is to provide a tool to solve the most common geometric transformation which are investigated in the context of the popular pinhole camera model. In more depth, the following code cells allows to convert the coordinates of a point:
- from image reference system (2D) to camera reference system (3D)
- from camera reference system (3D) to image reference system (2D)
- from camera reference system (3D) to real world reference system (3D)
- from real world reference system (3D) to camera reference system (3D)

**Note:** in this paper we assume that the 2D image place is a rectangle, i.e. the angle between the $X$ and $Y$ axis is 90 degrees. If this not the case we need to take into account the *skew* parameter. [This website](https://towardsdatascience.com/what-are-intrinsic-and-extrinsic-camera-parameters-in-computer-vision-7071b72fb8ec) is a super clear and useful reference. In summary the intrinsic camera matrix becomes:
<br><br>
$\begin{bmatrix}
f_x & s & c_x \\
0 & f_y & c_y \\
0 & 0 & 1 \\
\end{bmatrix}$
<br><br>
Such that $s$ is the skew coefficient which can be computed as:
<br><br>
$s = -f_x \cot{(\theta)}$ where $\theta$ is the angle between the $x$ and $y$ axis.

In [28]:
import numpy as np
import math

## Taxonomy
- `intrinsic_camera_matrix` is the intrinsic camera matrix defined as follows:
<br><br>
$M_C^I = \begin{bmatrix}
f_x & 0 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1 \\
\end{bmatrix}$

- `extrinsic_camera_matrix` is the extrinsic camera matrix defined as follows:
<br><br>
$M_W^C = \begin{bmatrix}
r_{00} & r_{01} & r_{02} & t_x \\
r_{10} & r_{11} & r_{12} & t_y \\
r_{20} & r_{21} & r_{22} & t_z \\
\end{bmatrix}$

- `scaling_factor` $s$ is the scaling factor (by default equal to 1).

- `image_coordinates` is a $2 \times 1$ vector which represent the coordinates of the point of interest in the image reference system (i.e. pixel coordinates).
<br><br>
$P_I = \begin{bmatrix}
u \\
v \\
\end{bmatrix}$

- `image_coordinates_h` is a $3 \times 1$ vector which represent the homogeneous coordinates of the point of interest in the image reference system (i.e. pixel coordinates).
<br><br>
$P_I = \begin{bmatrix}
su \\
sv \\
s
\end{bmatrix}$

- `camera_coordinates` is a $3 \times 1$ vector which represent the coordinates of the point of interest in the camera reference system.
<br><br>
$P_C = \begin{bmatrix}
x \\
y \\
z
\end{bmatrix}$

- `camera_coordinates_h` is a $4 \times 1$ vector which represent the homogeneous coordinates of the point of interest in the camera reference system.
<br><br>
$P_C = \begin{bmatrix}
x_C \\
y_C \\
z_C \\
1
\end{bmatrix}$

- `world_coordinates` is a $3 \times 1$ vector which represent the coordinates of the point of interest in the real world reference system.
<br><br>
$P_W = \begin{bmatrix}
x_W \\
y_W \\
z_W
\end{bmatrix}$

- `world_coordinates_h` is a $4 \times 1$ vector which represent the homogeneous coordinates of the point of interest in the real world reference system.
<br><br>
$P_W = \begin{bmatrix}
x_W \\
y_W \\
z_W \\
1
\end{bmatrix}$

## Utils
In this section we report some functions which might be useful as reference. For example, we provide the Python code to compute the inverse of a matrix.

### Matrix inverse

In [29]:
def matrix_inverse(input_matrix):
    return np.linalg.inv(input_matrix)

input_matrix = np.array([[2,1],[5,3]])
print(matrix_inverse(input_matrix))

[[ 3. -1.]
 [-5.  2.]]


### Matrix product

In [30]:
def matrix_product(matrix_a, matrix_b):
    return matrix_a.dot(matrix_b)

matrix_a = np.array([[2,1],[5,3]])
matrix_b = np.array([[3,-1],[-5,2]])
print(matrix_product(matrix_a, matrix_b))

[[1 0]
 [0 1]]


## Extrinsic camera matrix $M_W^C$

$M_W^C = \begin{bmatrix}
r_{00} & r_{01} & r_{02} & t_x \\
r_{10} & r_{11} & r_{12} & t_y \\
r_{20} & r_{21} & r_{22} & t_z \\
\end{bmatrix}$

### Rotation matrix

$R_{xyz} = R_x \times R_y \times R_z$

#### Roll matrix $R_x$

$R_x = \begin{bmatrix}
1 & 0 & 0 \\
0 & \cos{(\theta)} & -\sin{(\theta)} \\
0 & \sin{(\theta)} & \cos{(\theta)} \\
\end{bmatrix}$

In [31]:
def get_roll_matrix(roll_angle):
    roll_matrix = np.array([[1,0,0],
                            [0, np.cos(roll_angle), -np.sin(roll_angle)],
                            [0, np.sin(roll_angle), np.cos(roll_angle)]])
    return roll_matrix

roll_angle = np.radians(100)
roll_matrix = get_roll_matrix(roll_angle)
print(roll_matrix)

[[ 1.          0.          0.        ]
 [ 0.         -0.17364818 -0.98480775]
 [ 0.          0.98480775 -0.17364818]]


#### Pitch matrix $R_y$

$R_y = \begin{bmatrix}
\cos{(\theta)} & 0 & \sin{(\theta)} \\
0 & 1 & 0 \\
-\sin{(\theta)} & 0 & \cos{(\theta)} \\
\end{bmatrix}$

In [32]:
def get_pitch_matrix(pitch_angle):
    pitch_matrix = np.array([[np.cos(pitch_angle), 0, np.sin(pitch_angle)],
                            [0, 1, 0],
                            [-np.sin(pitch_angle), 0, np.cos(pitch_angle)]])
    return pitch_matrix

pitch_angle = np.radians(0)
pitch_matrix = get_pitch_matrix(pitch_angle)
print(pitch_matrix)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]]


#### Yaw matrix $R_z$

$R_z = \begin{bmatrix}
\cos{(\theta)} & -\sin{(\theta)} & 0 \\
\sin{(\theta)} & \cos{(\theta)} & 0 \\
0 & 0 & 1 \\
\end{bmatrix}$

In [33]:
def get_yaw_matrix(yaw_angle):
    yaw_matrix = np.array([ [np.cos(yaw_angle), -np.sin(yaw_angle), 0],
                            [np.sin(yaw_angle), np.cos(yaw_angle), 0],
                            [0,0,1]])
    return yaw_matrix

yaw_angle = np.radians(90)
yaw_matrix = get_yaw_matrix(yaw_angle)
print(yaw_matrix)

[[ 6.123234e-17 -1.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00]]


In [34]:
rotation_matrix = yaw_matrix.dot(pitch_matrix).dot(roll_matrix)
print(rotation_matrix)

[[ 6.12323400e-17  1.73648178e-01  9.84807753e-01]
 [ 1.00000000e+00 -1.06328842e-17 -6.03020831e-17]
 [ 0.00000000e+00  9.84807753e-01 -1.73648178e-01]]


In [35]:
# extrinsic camera matrix parameteres
r11 = rotation_matrix[0][0]
r12 = rotation_matrix[0][1]
r13 = rotation_matrix[0][2]
r21 = rotation_matrix[1][0]
r22 = rotation_matrix[1][1]
r23 = rotation_matrix[1][2]
r31 = rotation_matrix[2][0]
r32 = rotation_matrix[2][1]
r33 = rotation_matrix[2][2]

tx = 0.5
ty = 0.16
tz = 1.14

extrinsic_matrix = np.array([[r11, r12, r13, tx],
                             [r21, r22, r23, ty],
                             [r31, r32, r33, tz],
                             [0, 0, 0, 1]])

## Intrinsic camera matrix $M_C^I$

$M_C^I = \begin{bmatrix}
f_x & 0 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1 \\
\end{bmatrix}$

In [36]:
cx = 636
cy = 548
fx = 241
fy = 238

intrinsic_camera_matrix = np.array([[fx, 0, cx],
                                    [0, fy, cy],
                                    [0, 0, 1]])

## camera RS $\rightarrow$ image RS

$P^I = \begin{bmatrix}
su \\
sv \\
s
\end{bmatrix} = M_C^I P^C = \begin{bmatrix}
f_x & 0 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1 \\
\end{bmatrix} \begin{bmatrix}
x_C \\
y_C \\
z_C
\end{bmatrix}$

In [37]:
x_camera = 1.43026
y_camera = -0.73782
z_camera = 2.16788
camera_coordinates = np.array([[x_camera], [y_camera], [z_camera]])

image_coordinates_h = intrinsic_camera_matrix.dot(camera_coordinates)

s = image_coordinates_h[2][0]
u = image_coordinates_h[0][0]/s
v = image_coordinates_h[1][0]/s

print(f"u={u}")
print(f"v={v}")
print(f"s={s}")

u=794.9998800671625
v=466.99867151318335
s=2.16788


## image RS $\rightarrow$ camera RS 
$P^C = \begin{bmatrix}
x_C \\
y_C \\
z_C
\end{bmatrix} = (M_C^I)^{-1} P^I = \begin{bmatrix}
\frac{1}{f_x} & -\frac{1}{f_xf_y} & \frac{c_y-c_xf_y}{f_xf_y} \\
0 & \frac{1}{f_y} & -\frac{c_y}{f_y} \\
0 & 0 & 1 \\
\end{bmatrix} \begin{bmatrix}
su \\
sv \\
s
\end{bmatrix}$

In [38]:
u = 795
v = 467
scaling_factor = 2.16788

image_coordinates_h = np.array([[scaling_factor*u], [scaling_factor*v], [scaling_factor]])

camera_coordinates = np.linalg.inv(intrinsic_camera_matrix).dot(image_coordinates_h)

x_camera = camera_coordinates[0][0]
y_camera = camera_coordinates[1][0]
z_camera = camera_coordinates[2][0]

print(f"x_camera={x_camera}")
print(f"y_camera={y_camera}")
print(f"z_camera={z_camera}")

x_camera=1.4302610788381744
y_camera=-0.7378078991596629
z_camera=2.16788


## world RS $\rightarrow$ camera RS

In [39]:
x_world = 2.507
y_world = 1.590
z_world = 0.037

world_coordinates = np.array([[x_world], [y_world], [z_world]])
world_coordinates_h = np.append(world_coordinates, 1)

camera_coordinates_h = np.linalg.inv(extrinsic_matrix).dot(world_coordinates_h)
camera_coordinates = camera_coordinates_h[:-1]


x_camera = camera_coordinates[0]
y_camera = camera_coordinates[1]
z_camera = camera_coordinates[2]

print(f"x_camera={x_camera}")
print(f"y_camera={y_camera}")
print(f"z_camera={z_camera}")

x_camera=1.4300000000000004
y_camera=-0.7377310589949362
z_camera=2.1680431002621257


## camera RS $\rightarrow$ world RS

In [40]:
x_camera = 1.43026
y_camera = -0.73782
z_camera = 2.16788

camera_coordinates = np.array([[x_camera], [y_camera], [z_camera]])
camera_coordinates_h = np.append(camera_coordinates, 1)

world_coordinates_h = extrinsic_matrix.dot(camera_coordinates_h)
world_coordinates = world_coordinates_h[:-1]

x_world = world_coordinates[0]
y_world = world_coordinates[1]
z_world = world_coordinates[2]

print(f"x_world = {x_world}")
print(f"y_world = {y_world}")
print(f"z_world = {z_world}")

x_world = 2.506823933153891
y_world = 1.5902599999999998
z_world = 0.03694073227194772
